# Geodist Python usage

This notebook demos the public `geodist` Python API. It keeps to the exported surface
available from `import geodist` (no private Rust module access).

Prerequisites when running from the repo root:
- `cd pygeodist && uv sync --all-extras`
- `uv run maturin develop` to build the extension module before importing

Open the notebook from `pygeodist/` (or adjust the sys.path cell below) so imports resolve.


In [9]:
from __future__ import annotations

import pathlib
import sys

# Allow running the notebook from pygeodist/notebooks while using the local source tree.
project_root = pathlib.Path('..').resolve()
python_src = project_root / 'src'
if python_src.is_dir() and str(python_src) not in sys.path:
    sys.path.insert(0, str(python_src))

print(f'Using Python {sys.version.split()[0]}')
print(f'Notebook cwd: {pathlib.Path.cwd()}')
print(f'Import path head: {sys.path[0]}')


Using Python 3.11.11
Notebook cwd: /home/ejones/workspace/eddieland/geodist/pygeodist/notebooks
Import path head: /home/ejones/.local/share/uv/python/cpython-3.11.11-linux-x86_64-gnu/lib/python311.zip


In [10]:
from geodist import (
    EARTH_RADIUS_METERS,
    BoundingBox,
    Ellipsoid,
    GeodesicResult,
    GeodistError,
    InvalidGeometryError,
    Point,
    Point3D,
    geodesic_distance,
    geodesic_distance_on_ellipsoid,
    geodesic_distance_3d,
    geodesic_with_bearings,
    geodesic_with_bearings_on_ellipsoid,
    hausdorff,
    hausdorff_3d,
    hausdorff_directed,
    hausdorff_directed_3d,
    hausdorff_clipped,
    hausdorff_clipped_3d,
    hausdorff_directed_clipped,
    hausdorff_directed_clipped_3d,
)

EARTH_RADIUS_METERS


6371008.8

## Visualization helpers

The snippets below render small inline SVG maps to make Hausdorff witness pairs and
clipping envelopes easier to reason about without pulling in heavier plotting
libraries.
    


In [ ]:
from typing import Sequence

from IPython.display import SVG


def _lat_lon_alt(points: Sequence[Point | Point3D]):
    coords: list[tuple[float, float, float | None]] = []
    for point in points:
        coords.append((point.lat, point.lon, getattr(point, "altitude_m", None)))
    return coords


def render_witness_plot(
    points_a: Sequence[Point | Point3D],
    points_b: Sequence[Point | Point3D],
    *,
    symmetric_witness: HausdorffWitness | None = None,
    directed_witness: HausdorffDirectedWitness | None = None,
    bounding_box: BoundingBox | None = None,
    title: str = "Witness points",
    width: int = 720,
    height: int = 420,
):
    '''Render a simple SVG diagram highlighting witness pairs and bounding boxes.'''

    coords_a = _lat_lon_alt(points_a)
    coords_b = _lat_lon_alt(points_b)

    lats = [lat for lat, _, _ in coords_a + coords_b]
    lons = [lon for _, lon, _ in coords_a + coords_b]
    if bounding_box is not None:
        lats.extend([bounding_box.min_lat, bounding_box.max_lat])
        lons.extend([bounding_box.min_lon, bounding_box.max_lon])

    min_lat, max_lat = min(lats), max(lats)
    min_lon, max_lon = min(lons), max(lons)
    lat_span = max(max_lat - min_lat, 1e-6)
    lon_span = max(max_lon - min_lon, 1e-6)
    padding = 30

    def project(lat: float, lon: float) -> tuple[float, float]:
        x = padding + ((lon - min_lon) / lon_span) * (width - 2 * padding)
        y = height - padding - ((lat - min_lat) / lat_span) * (height - 2 * padding)
        return x, y

    svg_parts = [
        f'<svg width="{width}" height="{height}" viewBox="0 0 {width} {height}" ',
        'xmlns="http://www.w3.org/2000/svg" role="img">',
        f'<text x="{padding}" y="{padding - 8}" font-family="Inter, sans-serif"',
        ' font-size="16" font-weight="600">' + title + '</text>',
    ]

    if bounding_box is not None:
        x0, y1 = project(bounding_box.min_lat, bounding_box.min_lon)
        x1, y0 = project(bounding_box.max_lat, bounding_box.max_lon)
        svg_parts.append(
            f'<rect x="{min(x0, x1):.2f}" y="{min(y0, y1):.2f}" '
            f'width="{abs(x1 - x0):.2f}" height="{abs(y1 - y0):.2f}" '
            'fill="rgba(64, 160, 255, 0.08)" stroke="#40a0ff" '
            'stroke-dasharray="6 4" stroke-width="2" rx="6" />'
        )
        svg_parts.append(
            f'<text x="{min(x0, x1) + 6:.2f}" y="{min(y0, y1) + 18:.2f}" '
            'font-family="Inter, sans-serif" font-size="12" fill="#1f5073">'
            'Clipping envelope</text>'
        )

    legend_y = height - padding + 4
    svg_parts.extend(
        [
            f'<circle cx="{padding}" cy="{legend_y}" r="6" fill="#2a6fdb" />',
            f'<text x="{padding + 12}" y="{legend_y + 4}" font-family="Inter, sans-serif" '
            'font-size="12">Set A</text>',
            f'<circle cx="{padding + 90}" cy="{legend_y}" r="6" fill="#e6862a" />',
            f'<text x="{padding + 102}" y="{legend_y + 4}" font-family="Inter, sans-serif" '
            'font-size="12">Set B</text>',
        ]
    )

    def draw_points(coords, color, label_prefix):
        for idx, (lat, lon, altitude) in enumerate(coords):
            x, y = project(lat, lon)
            svg_parts.append(
                f'<circle cx="{x:.2f}" cy="{y:.2f}" r="6" fill="{color}" stroke="#0f172a" stroke-width="0.8" />'
            )
            svg_parts.append(
                f'<text x="{x + 9:.2f}" y="{y + 4:.2f}" font-family="Inter, sans-serif" font-size="12" fill="#0f172a">'
                f'{label_prefix}{idx}</text>'
            )
            if altitude is not None:
                svg_parts.append(
                    f'<text x="{x + 9:.2f}" y="{y - 10:.2f}" font-family="Inter, sans-serif" font-size="11" fill="#475569">'
                    f'{altitude:.0f} m</text>'
                )

    draw_points(coords_a, "#2a6fdb", "A")
    draw_points(coords_b, "#e6862a", "B")

    def draw_witness(label: str, witness: HausdorffDirectedWitness, origin_is_a: bool):
        origin_coords = coords_a if origin_is_a else coords_b
        candidate_coords = coords_b if origin_is_a else coords_a
        try:
            origin = origin_coords[witness.origin_index]
            candidate = candidate_coords[witness.candidate_index]
        except IndexError:
            return

        x0, y0 = project(origin[0], origin[1])
        x1, y1 = project(candidate[0], candidate[1])
        color = "#d63d3d" if origin_is_a else "#7a3dd6"
        svg_parts.append(
            f'<line x1="{x0:.2f}" y1="{y0:.2f}" x2="{x1:.2f}" y2="{y1:.2f}" '
            f'stroke="{color}" stroke-width="3" stroke-linecap="round" opacity="0.9" />'
        )
        mid_x, mid_y = (x0 + x1) / 2, (y0 + y1) / 2
        svg_parts.append(
            f'<text x="{mid_x + 6:.2f}" y="{mid_y - 6:.2f}" font-family="Inter, sans-serif" '
            'font-size="12" fill="#0f172a">' +
            f'{label}: {witness.distance_m:.1f} m</text>'
        )

    if symmetric_witness is not None:
        draw_witness("A→B", symmetric_witness.a_to_b, origin_is_a=True)
        draw_witness("B→A", symmetric_witness.b_to_a, origin_is_a=False)

    if directed_witness is not None:
        draw_witness("Directed", directed_witness, origin_is_a=True)

    svg_parts.append("</svg>")
    return SVG("".join(svg_parts))
    


## Points and tuples

Create latitude/longitude points (degrees) and optional altitude for 3D points.
Each wrapper can round-trip to tuples for interoperability.


In [11]:
nyc = Point(40.7128, -74.0060)
london = Point(51.5074, -0.1278)

print('NYC tuple:', nyc.to_tuple())
print('London tuple:', london.to_tuple())
print('NYC repr:', nyc)

# 3D point with altitude in meters
viewpoint = Point3D(37.7335, -119.5580, 2693.0)
print('3D point:', viewpoint)


NYC tuple: (40.7128, -74.006)
London tuple: (51.5074, -0.1278)
NYC repr: Point(lat=40.7128, lon=-74.006)
3D point: Point3D(lat=37.7335, lon=-119.558, altitude_m=2693.0)


## Great-circle distance and bearings

Use `geodesic_distance` for the spherical fast path (mean-radius approximation) and
`geodesic_with_bearings` for distance plus initial/final bearings.


In [12]:
nyc_to_london_m = geodesic_distance(nyc, london)
bearings: GeodesicResult = geodesic_with_bearings(nyc, london)

print(f'NYC to London: {nyc_to_london_m/1000:.1f} km')
print(f'  initial bearing: {bearings.initial_bearing_deg:.1f} deg')
print(f'  final bearing:   {bearings.final_bearing_deg:.1f} deg')


NYC to London: 5570.2 km
  initial bearing: 51.2 deg
  final bearing:   108.3 deg


## Ellipsoidal geodesics (WGS84 or custom)

Use `geodesic_distance_on_ellipsoid` / `geodesic_with_bearings_on_ellipsoid` for
higher-accuracy results tied to a specific ellipsoid. WGS84 is the default; provide
explicit axes when working with alternate datums or planets.


In [ ]:
wgs84 = Ellipsoid.wgs84()
spherical_nyc_london_m = geodesic_distance(nyc, london)
ellipsoidal_nyc_london_m = geodesic_distance_on_ellipsoid(nyc, london, wgs84)
ellipsoidal_bearings = geodesic_with_bearings_on_ellipsoid(nyc, london, wgs84)

print(f'Spherical mean-radius: {spherical_nyc_london_m/1000:.2f} km')
print(f'WGS84 ellipsoidal:     {ellipsoidal_nyc_london_m/1000:.2f} km')
print(
    f'Bearings (init/final): '
    f'{ellipsoidal_bearings.initial_bearing_deg:.2f} deg / '
    f'{ellipsoidal_bearings.final_bearing_deg:.2f} deg'
)

mars_semi_major_m = 3_396_190.0
mars_flattening = 1.0 / 169.8944472
mars_semi_minor_m = mars_semi_major_m * (1.0 - mars_flattening)
mars = Ellipsoid(mars_semi_major_m, mars_semi_minor_m)
pathfinder = Point(19.26, 326.75)
curiosity = Point(-4.765700445, 137.39820983)
mars_distance_m = geodesic_distance_on_ellipsoid(curiosity, pathfinder, mars)
print(f'Mars pathfinder->curiosity distance: {mars_distance_m/1000:.2f} km')


## 3D straight-line distance (ECEF chord)

`geodesic_distance_3d` computes a straight-line chord through space using
latitude/longitude in degrees and altitude in meters.


In [13]:
# Yosemite Valley viewpoint to Yosemite Falls overlook (approximate)
valley_floor = Point3D(37.7331, -119.5586, 1200.0)
clifftop = Point3D(37.7335, -119.5580, 2693.0)

line_of_sight_m = geodesic_distance_3d(valley_floor, clifftop)
print(f'Line-of-sight distance: {line_of_sight_m:.1f} m')


Line-of-sight distance: 1494.6 m


## Hausdorff distance over point sets

Hausdorff compares two sets of points; directed answers
"how far is set A from being covered by B?" while symmetric checks the max distance either direction.


In [14]:
trail_a = [
    Point(47.6205, -122.3493),
    Point(47.6220, -122.3470),
    Point(47.6235, -122.3440),
]
trail_b = [
    Point(47.6200, -122.3490),
    Point(47.6215, -122.3465),
    Point(47.6230, -122.3435),
]

symmetric_witness = hausdorff(trail_a, trail_b)
directed_ab_witness = hausdorff_directed(trail_a, trail_b)
directed_ba_witness = hausdorff_directed(trail_b, trail_a)

print(f'Symmetric Hausdorff: {symmetric_witness.distance_m:.2f} m')
print(
    f'Directed (A -> B): {directed_ab_witness.distance_m:.2f} m from '
    f'A[{directed_ab_witness.origin_index}] -> B[{directed_ab_witness.candidate_index}]'
)
print(
    f'Directed (B -> A): {directed_ba_witness.distance_m:.2f} m from '
    f'B[{directed_ba_witness.origin_index}] -> A[{directed_ba_witness.candidate_index}]'
)


Symmetric Hausdorff: 67.05 m
Directed A->B:       67.05 m
Directed B->A:       67.05 m


In [ ]:
witness_plot = render_witness_plot(
    points_a=trail_a,
    points_b=trail_b,
    symmetric_witness=symmetric_witness,
    title="Trail overlap witness pairs",
)
witness_plot
            


## Bounding-box clipping to ignore outliers

Clipped variants restrict evaluation to a latitude/longitude envelope,
which is useful when one set contains distant outliers you want to ignore.


In [15]:
outlier = Point(10.0, 10.0)
trail_b_with_outlier = trail_b + [outlier]

bbox = BoundingBox(47.61, 47.64, -122.36, -122.33)

raw_witness = hausdorff(trail_a, trail_b_with_outlier)
clipped_witness = hausdorff_clipped(trail_a, trail_b_with_outlier, bbox)

print(f'Hausdorff with outlier: {raw_witness.distance_m/1000:.2f} km')
print(f'Clipped Hausdorff:      {clipped_witness.distance_m:.2f} m')


Hausdorff with outlier: 12074.82 km
Clipped Hausdorff:      67.05 m


In [ ]:
render_witness_plot(
    points_a=trail_a,
    points_b=trail_b_with_outlier,
    symmetric_witness=raw_witness,
    bounding_box=bbox,
    title="Outlier pushing the Hausdorff distance",
)
            


## 3D Hausdorff distances for altitude-aware tracks

When trail segments vary in altitude, the 3D Hausdorff helpers compare ECEF
coordinates so vertical separation counts toward the distance. Use the directed
variants to see how far each path deviates from the other and the symmetric
result to summarize both directions.


In [ ]:
ridge_a = [
    Point3D(37.7331, -119.5586, 1200.0),
    Point3D(37.7340, -119.5570, 1500.0),
    Point3D(37.7350, -119.5550, 2100.0),
]
ridge_b = [
    Point3D(37.7332, -119.5584, 1195.0),
    Point3D(37.7342, -119.5568, 1510.0),
    Point3D(37.7348, -119.5552, 2080.0),
]

ridge_symmetric_witness = hausdorff_3d(ridge_a, ridge_b)
ridge_directed_ab_witness = hausdorff_directed_3d(ridge_a, ridge_b)
ridge_directed_ba_witness = hausdorff_directed_3d(ridge_b, ridge_a)

print(f'3D symmetric Hausdorff: {ridge_symmetric_witness.distance_m:.2f} m')
print(
    f'Directed (A -> B): {ridge_directed_ab_witness.distance_m:.2f} m from '
    f'A[{ridge_directed_ab_witness.origin_index}] -> B[{ridge_directed_ab_witness.candidate_index}]'
)
print(
    f'Directed (B -> A): {ridge_directed_ba_witness.distance_m:.2f} m from '
    f'B[{ridge_directed_ba_witness.origin_index}] -> A[{ridge_directed_ba_witness.candidate_index}]'
)


In [ ]:
render_witness_plot(
    points_a=ridge_a,
    points_b=ridge_b,
    symmetric_witness=ridge_symmetric_witness,
    title="Ridge comparison with altitude labels",
)
            


### Clipping 3D paths to ignore unrelated detours

The clipping envelope applies to the horizontal latitude/longitude bounds while
still accounting for altitude in the distance calculation. This helps isolate
sections of a trail without being skewed by faraway diversions.


In [ ]:
ridge_outlier = ridge_b + [Point3D(37.8000, -119.7000, 500.0)]
ridge_bbox = BoundingBox(37.73, 37.74, -119.56, -119.55)

ridge_raw_witness = hausdorff_3d(ridge_a, ridge_outlier)
ridge_clipped_witness = hausdorff_clipped_3d(ridge_a, ridge_outlier, ridge_bbox)

print(f'3D Hausdorff with detour: {ridge_raw_witness.distance_m/1000:.2f} km')
print(f'3D Hausdorff clipped:     {ridge_clipped_witness.distance_m:.2f} m')


In [ ]:
render_witness_plot(
    points_a=ridge_a,
    points_b=ridge_outlier,
    symmetric_witness=ridge_raw_witness,
    bounding_box=ridge_bbox,
    title="Clipping away a far 3D detour",
)
            


## Error handling and validation

Geometry inputs are validated before computation. Invalid coordinates raise
`InvalidGeometryError`, while empty inputs surface as `GeodistError`. Catch
these exceptions to provide clearer user-facing messages in data pipelines.


In [ ]:
try:
    Point(95.0, 0.0)
except InvalidGeometryError as exc:
    print(f'Invalid geometry: {exc}')

try:
    hausdorff([], trail_b)
except GeodistError as exc:
    print(f'Empty input error: {exc}')
